<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/basics/export_data.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/basics/export_data.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

Taken and modified from: https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/basics/export_data.ipynb

In [26]:
import urllib.request
from pathlib import Path

import labelbox as lb
import yaml
from PIL import Image

In [27]:
config = yaml.safe_load(open("./config.yml"))

client = lb.Client(api_key=config["api_key"])

save_dir = Path("./annotated")
save_dir.mkdir(parents=True, exist_ok=True)

PROJECT_ID = config["project_id"]
project = client.get_project(PROJECT_ID)

In [28]:
# Set the export params to include/exclude certain fields.
export_params = {
    "attachments": True,
    "metadata_fields": True,
    "data_row_details": True,
    "project_details": True,
    "label_details": True,
    "performance_details": True,
    "interpolated_frames": True,
}

export_task = project.export_v2(params=export_params)
export_task.wait_till_done()

if export_task.errors:
    print(export_task.errors)

export_json = export_task.result

In [29]:
feature_ids = [
    (
        row["data_row"]["external_id"],
        row["projects"][PROJECT_ID]["labels"][0]["annotations"]["objects"][0][
            "feature_id"
        ],
    )
    for row in export_json
]
mask_urls = [
    f"https://api.labelbox.com/api/v1/projects/{PROJECT_ID}/annotations/{feature_id[1]}/index/1/mask"
    for feature_id in feature_ids
]

In [30]:
for mask_url, feature_id in zip(mask_urls, feature_ids):
    # Make the API request
    req = urllib.request.Request(mask_url, headers=client.headers)
    # Print the image of the mask
    image = Image.open(urllib.request.urlopen(req))
    name = feature_id[0].replace(".jpg", ".png")
    image.save(save_dir / name, "png")